# QUERIES

## SQL

A continuación presentamos el modelo entidad relación:

![Modelo entidad relación](diagrama_er.png)

In [1]:
import pandas as pd
from sqlalchemy import create_engine

# Conexión a PostgreSQL
user = "userGESTDB"
password = "passGESTDB"
host = "postgres_db" 
port = "5432"
db = "GESTDB"

engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{db}")

df = pd.read_sql("SELECT * FROM grado;", engine)

In [2]:
df.fillna({
        'nombre': 'Unknown', 
        'descripcion':'Este grado no contiene descripción',
        'salidas':'Este grado no contiene salidas'
        },
    inplace=True)

A. Ver grados con su rama, área y universidad

In [3]:
df_1 = pd.read_sql(
"""
SELECT  
    g.id AS id_grado, 
    g.nombre AS grado, 
    a.nombre AS area, 
    r.nombre AS rama, 
    u.nombre AS universidad 

FROM grado g 
JOIN area a ON g.id_area = a.id 
JOIN rama r ON a.id_rama = r.id 
JOIN grado_ofertado go ON g.id = go.id_grado 
JOIN facultad f ON go.id_facultad = f.id 
JOIN universidad u ON f.id_universidad = u.id 
ORDER BY r.nombre, g.nombre;
""", engine)
df_1.head()

,id_grado,grado,area,rama,universidad
0,52,Antropología Social y Cultural,Antropología Social y Cultural,Artes y Humanidades,Universidad Europea de Madrid
1,1,Antropología Social y Cultural,Antropología Social y Cultural,Artes y Humanidades,Universidad Autónoma de Madrid
2,1,Antropología Social y Cultural,Antropología Social y Cultural,Artes y Humanidades,Universidad Nacional de Educación a Distancia ...
3,1,Antropología Social y Cultural,Antropología Social y Cultural,Artes y Humanidades,Universidad Complutense de Madrid
4,32,Arqueología,Geografía e Historia,Artes y Humanidades,Universidad Nacional de Educación a Distancia ...


B. Consulta de titulaciones posibles según la nota de admisión

In [4]:
df_2 = pd.read_sql(
    """
SELECT  
    g.nombre AS grado, 
    u.nombre AS universidad, 
    nc.nota AS nota_corte, 
    nc.año, 
    nc.convocatoria 
FROM nota_corte nc 
JOIN grado_ofertado go ON nc.id_grado_ofertado = go.id 
JOIN grado g ON go.id_grado = g.id 
JOIN facultad f ON go.id_facultad = f.id 
JOIN universidad u ON f.id_universidad = u.id 
WHERE nc.nota <= 11.0 

ORDER BY nc.nota DESC; 
    """, engine)
df_2.head()

,grado,universidad,nota_corte,año,convocatoria
0,Ingeniería en Tecnologías Industriales,Universidad Carlos III de Madrid,11.0,2025,ordinaria
1,Ingeniería en Tecnologías Industriales,Universidad a Distancia de Madrid UDIMA,11.0,2025,ordinaria
2,Ingeniería en Tecnologías Industriales,Universidad de Alcalá,11.0,2025,ordinaria
3,Ingeniería en Tecnologías Industriales,Universidad Politécnica de Madrid,11.0,2025,ordinaria
4,Ingeniería en Tecnologías Industriales,Universidad Pontificia Comillas ICAIICADE,11.0,2025,ordinaria


C. Consulta de universidades públicas por rama de conocimiento

In [5]:
df_3 = pd.read_sql(
    """
SELECT DISTINCT  
    u.nombre AS universidad, 
    r.nombre AS rama 

FROM universidad u 
JOIN facultad f ON u.id = f.id_universidad 
JOIN grado_ofertado go ON f.id = go.id_facultad 
JOIN grado g ON go.id_grado = g.id 
JOIN area a ON g.id_area = a.id 
JOIN rama r ON a.id_rama = r.id 

WHERE u.tipo = 'Pública' 
  AND r.nombre = 'Ingeniería y Arquitectura' 
ORDER BY u.nombre; 
    """, engine)
df_3.head()

,universidad,rama
0,Universidad Autónoma de Madrid,Ingeniería y Arquitectura
1,Universidad Carlos III de Madrid,Ingeniería y Arquitectura
2,Universidad Complutense de Madrid,Ingeniería y Arquitectura
3,Universidad de Alcalá,Ingeniería y Arquitectura
4,Universidad Nacional de Educación a Distancia ...,Ingeniería y Arquitectura


D. Comparativa de grados similares según nota media

In [6]:
df_4 = pd.read_sql(
    """
SELECT  
    a.nombre AS area, 
    g.nombre AS grado, 
    ROUND(AVG(nc.nota), 2) AS nota_media 

FROM grado g 
JOIN area a ON g.id_area = a.id 
JOIN grado_ofertado go ON g.id = go.id_grado 
JOIN nota_corte nc ON go.id = nc.id_grado_ofertado 
GROUP BY a.nombre, g.nombre 

ORDER BY a.nombre, nota_media DESC; 
    """, engine)
df_4.head()

,area,grado,nota_media
0,Antropología Social y Cultural,Antropología Social y Cultural,5.00
1,Arquitectura,Fundamentos de Arquitectura y Urbanismo,8.64
2,Artes,Conservación y Restauración del Patrimonio Cul...,6.71
3,Artes,Bellas Artes,6.32
4,Artes,Composición Musical,5.00


E. Consultar grados de una misma facultad

In [7]:
df_5 = pd.read_sql(
    """
    SELECT  
        g.id AS id_grado, 
        g.nombre AS grado,
        u.nombre AS universidad,
        f.nombre AS factultad
    FROM universidad u
    JOIN facultad f ON f.id_universidad = u.id 
    JOIN grado_ofertado go ON go.id_facultad = f.id
    JOIN grado g ON g.id = go.id_grado
    WHERE f.id= 9
    ORDER BY g.nombre;
    """,
    engine
)

df_5.head()


,id_grado,grado,universidad,factultad
0,108,Bachelor of Urban Management,Universidad San PabloCEU,Facultad de Humanidades y Ciencias de la Comun...
1,109,Estudios Urbanos,Universidad San PabloCEU,Facultad de Humanidades y Ciencias de la Comun...
2,33,Geografía e Historia,Universidad San PabloCEU,Facultad de Humanidades y Ciencias de la Comun...
3,34,Historia,Universidad San PabloCEU,Facultad de Humanidades y Ciencias de la Comun...
4,111,Información y Documentación,Universidad San PabloCEU,Facultad de Humanidades y Ciencias de la Comun...


F. Consultar facultades de una universidad

In [8]:
df_6 = pd.read_sql(
    """
    SELECT  
        u.nombre AS universidad,
        f.nombre AS factultad
    FROM universidad u
    JOIN facultad f ON f.id_universidad = u.id 
    WHERE u.id= 1
    ORDER BY f.nombre;
    """,
    engine
)

df_6.head()


,universidad,factultad
0,Universidad Nacional de Educación a Distancia ...,Escuela Técnica Superior de Ingenieros Industr...
1,Universidad Nacional de Educación a Distancia ...,Facultad Ciencias Económicas y Empresariales
2,Universidad Nacional de Educación a Distancia ...,Facultad de Ciencias
3,Universidad Nacional de Educación a Distancia ...,Facultad de Ciencias Económicas y Empresariales
4,Universidad Nacional de Educación a Distancia ...,Facultad de Ciencias Políticas y Sociología


## Elasticsearch

In [9]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://elasticsearch:9200")

# Verifica la conexión
print(es.info().body)

{'name': '421114b78899', 'cluster_name': 'docker-cluster', 'cluster_uuid': '0LnWcaY3Tyi6oDYpU1mIgA', 'version': {'number': '8.7.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '09520b59b6bc1057340b55750186466ea715e30e', 'build_date': '2023-03-27T16:31:09.816451435Z', 'build_snapshot': False, 'lucene_version': '9.5.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [10]:
# indice semántico

index_mapping = {
    "properties": {
        "id": { "type": "integer" },        
        "nombre": { "type": "text" },     
        "id_area": { "type": "integer" },
        "descripcion": { "type": "text" },
        "descripcion_vector": {
            "type": "dense_vector",
            "dims": 384,  # Dimensionality of the embeddings
            "index": True,
            "similarity": "cosine"
        },
        "salidas": { "type": "text" },
        "salidas_vector": {
            "type": "dense_vector",
            "dims": 384,  # Dimensionality of the embeddings
            "index": True,
            "similarity": "cosine"
        }
    }
}

index_name = "informacion_grados"
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)
es.indices.create(
    index=index_name,
    mappings=index_mapping
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'informacion_grados'})

In [11]:
# creamos los embedings de la descripcion de titulo
from sentence_transformers import SentenceTransformer

# Load a pre-trained Sentence Transformer model
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Generate embeddings (vector representations) for descripciones
embeddings_descripcion = model.encode(df["descripcion"].values.tolist())

# Generate embeddings (vector representations) for salidas
embeddings_salidas = model.encode(df["salidas"].values.tolist())

/opt/conda/lib/python3.8/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [12]:
import json
from elasticsearch.helpers import bulk

index_name="informacion_grados"

def vector_bulk_index_data(es, data, index_name):
    batch_size = 50  # Reducir el tamaño del lote a 50
    for i in range(0, len(data), batch_size):
        batch = data[i:i+batch_size]
        actions = []
        for idx, doc in enumerate(batch):
            doc["descripcion_vector"]=embeddings_descripcion[i + idx].tolist()
            doc["salidas_vector"]=embeddings_salidas[i + idx].tolist()
            actions.append({
                "_index": index_name,
                "_id": doc['id'],
                "_source": doc
            })
        # Capturar la respuesta para verificar errores
        resp = bulk(es, actions, raise_on_error=True)
        print("Indexed:", resp[0], "Errors:", resp[1])

descripciones = df.to_dict(orient='records')
vector_bulk_index_data(es, descripciones, index_name)

Indexed: 50 Errors: []
Indexed: 50 Errors: []
Indexed: 50 Errors: []
Indexed: 50 Errors: []
Indexed: 29 Errors: []


### Queries sintácticas

A. Consulta de grados que contengan en el nombre *industrial*

In [29]:
import json

res = es.search(
    index=index_name,
    size=3,
    query={
        "term": {
            "nombre": "industrial"
        }
    }
)

print("Search Results:")
for hit in res['hits']['hits']:
     print(f"Document ID: {hit['_id']}, Nombre Título: {hit['_source']['nombre']}, Description: {hit['_source']['descripcion']}, Score: {hit['_score']}")
     print("")

Search Results:
Document ID: 190, Nombre Título: Ingeniería en Organización Industrial, Description: Un Ingeniero de Organización analiza y resuelve problemas para optimizar procesos empresariales, identificando prioridades estratégicas, recursos y beneficios. Su formación dual le permite tener una visión global de la empresa., Score: 4.1358886

Document ID: 192, Nombre Título: Ingeniería en Electrónica y Automática Industrial, Description: La titulación se centra en la tecnología punta y la creación de ingenieros mecánicos que mejoren producción y trabajo humano. Se aplican conocimientos a sistemas para mejorar procesos industriales, pero también se utilizan en medicina, agricultura, distribución de mercancías y energía., Score: 3.4236507

Document ID: 189, Nombre Título: Ingeniería en Diseño Industrial y Desarrollo de Producto, Description: La carrera de Diseño Industrial forma profesionales que diseñan productos innovadores, competitivos y eficientes desde el punto de vista funciona

B. Consulta de grados que contengan en el nombre *informática*

In [28]:
res = es.search(
    index=index_name,
    size=3,
    query={
        "term": {
            "nombre": "informática"
        }
    }
)

print("Search Results:")
for hit in res['hits']['hits']:
     print(f"Document ID: {hit['_id']}, Nombre Título: {hit['_source']['nombre']}, Description: {hit['_source']['descripcion']}, Score: {hit['_score']}")
     print("")

Search Results:
Document ID: 214, Nombre Título: Ingeniería Informática, Description: La carrera de Ingeniería Informática forma profesionales que crean programas informáticos ajustados a las necesidades sociales y empresariales. Los ingenieros informáticos utilizan lenguajes de programación para analizar problemas cotidianos y desarrollar soluciones lógicas. La titulación se centra en la gestión y explotación de sistemas informáticos, incluyendo el diseño y desarrollo de software orientado al usuario final., Score: 5.2223105

Document ID: 215, Nombre Título: Ingeniería Informática Biomédica, Description: El grado en Ingeniería Informática Biomédica se enfoca en la aplicación de tecnología, informática y programación en salud y biomedicina., Score: 4.616037

Document ID: 220, Nombre Título: Matemáticas e Informática, Description: Este grado combina Matemáticas e Informática, enfatizando la interrelación entre ambas. Forma graduados con conocimientos generales en ambos campos para acced

### Queries léxcias

C. Consulta de grados que devuelve grados que contengan en la descripción similares a *computación*

In [30]:
res = es.search(
    index="informacion_grados",
    size=3,
    query={
        "match": {
            "descripcion": "computación"
        }
    }
)


print("Search Results:")
for hit in res['hits']['hits']:
     print(f"Document ID: {hit['_id']}, Nombre Título: {hit['_source']['nombre']}, Description: {hit['_source']['descripcion']}, Score: {hit['_score']}")
     print("")

Search Results:
Document ID: 203, Nombre Título: Matemática Computacional, Description: El Grado en Matemática Aplicada y Computación forma profesionales que aplican matemáticas en áreas como la ciencia, ingeniería y economía. Los estudiantes aprenden algoritmos, métodos numéricos y programación para resolver problemas reales., Score: 4.7130547

Document ID: 225, Nombre Título: Ciencia de Datos e Inteligencia Artificial, Description: El grado forma profesionales versátiles con sólida base en Matemáticas, Estadística y Computación. Los egresados pueden aplicar habilidades en tecnologías de la información, ciencia de datos e inteligencia artificial para diseñar estrategias de gestión de datos y sistemas de información. También deben considerar aspectos éticos, legales y sostenibles en el uso de técnicas de IA y datos., Score: 3.658999



D. Consulta de grados que devuelve grados que mencionen en la descripción algo similar a *comunicación* y en salidas *medios audiovisuales* pero no tenga similitud en la descripción con *ingeniería* y *matemáticas*

In [16]:
res = es.search(
    index="informacion_grados",
    size=3,
    query={
        "bool": {
            "must": [
                {"match": {"descripcion": "comunicación"}},
                {"match": {"salidas": "medios audiovisuales"}}
            ],
            "must_not": [
                {"match": {"descripcion": "ingeniería"}},
                {"match": {"descripcion": "matemáticas"}}
            ]
        }
    }
)

print("Search Results:")
for hit in res['hits']['hits']:
     print(f"Document ID: {hit['_id']}, Nombre Título: {hit['_source']['nombre']}, Description: {hit['_source']['descripcion']}, Score: {hit['_score']}")
     print("")

Search Results:
Document ID: 126, Nombre Título: Comunicación y Medios Digitales, Description: El Grado en Periodismo forma a universitarios para realizar la función informativa con rigor, profundidad e idoneidad. Los estudiantes aprenden a transmitir noticias, redactar correctamente y expresarse en el lenguaje adecuado dependiendo del tipo de noticia y medio de comunicación. El objetivo es formar periodistas capacitados para trabajar en empresas relacionadas con la información y la comunicación., Score: 5.813483

Document ID: 123, Nombre Título: Diseño Digital y Multimedia, Description: El grado forma a diseñadores altamente cualificados en tecnología, comunicación y arte. Los estudiantes deben ser creativos, permeables al cambio y tener deseos de aprender permanentemente., Score: 5.761183

Document ID: 16, Nombre Título: Diseño, Description: El Grado en Diseño combina teoría y práctica con otras disciplinas como Arte, Arquitectura y Comunicación. Ofrece cuatro menciones: Diseño gráfi

E. Consulta de grados que devuelve grados que mencionen en la descripción algo similar a *gestión* y tenga saliddas relacionadas con *administración de empresas* pero que no tenga relación con *finanzas* y salidas de *contabilidad*

In [17]:
res = es.search(
    index="informacion_grados",
    size=1,
    query={
        "bool": {
            "must": [
                {"match": {"descripcion": "gestión"}},
                {"match": {"salidas": "administración de empresas"}}
            ],
            "must_not": [
                {"match": {"descripcion": "finanzas"}},
                {"match": {"salidas": "contabilidad"}}
            ]
        }
    }
)

print("Search Results:")
for hit in res['hits']['hits']:
     print(f"Document ID: {hit['_id']}, Nombre Título: {hit['_source']['nombre']}, Description: {hit['_source']['descripcion']}, Score: {hit['_score']}")
     print("")

Search Results:
Document ID: 56, Nombre Título: Gestión Deportiva, Description: El Grado en Gestión Deportiva forma a profesionales en turismo deportivo, marketing deportivo, gestión de centros deportivos y asesoría de instalaciones deportivas., Score: 6.2615194



F. Consulta de grados que devuelve grados que estén relacionados con *comportamiento humano* y tenga salidas relacionadas con *social* pero que no sean los grados *Psicología* ni *Biología Sanitaria* 

In [18]:
res = es.search(
    index="informacion_grados",
    size=3,
    query={
        "bool": {
            "must": [
                {"match": {"descripcion": "comportamiento humano"}},
                {"match": {"salidas": "social"}}
            ],
            "must_not": [
                {"term": {"nombre.keyword": "Psicología"}},
                {"term": {"nombre.keyword": "Biología Sanitaria"}}
            ]
        }
    }
)

print("Search Results:")
for hit in res['hits']['hits']:
    print(f"Document ID: {hit['_id']}, Nombre Título: {hit['_source']['nombre']}, Description: {hit['_source']['descripcion']}, Score: {hit['_score']}")
    print("")

Search Results:
Document ID: 91, Nombre Título: Comportamiento y Ciencias Sociales, Description: Este programa multidisciplinario aborda la comprensión del comportamiento humano desde diferentes ámbitos sociales. No es un Grado de Psicología, sino una formación holística que combina Ciencias Sociales para analizar y influir en el comportamiento humano. Se enfoca en desarrollar profesionales capaces de ayudar a empresas y organizaciones a ser más competitivas y centradas en el cliente., Score: 12.153459

Document ID: 61, Nombre Título: Ciencias Políticas, Description: El grado en Ciencias Políticas ofrece una formación pluridisciplinar centrada en el análisis de los procesos políticos contemporáneos. Se incluyen conocimientos teóricos y prácticos sobre comunicación política, comportamiento electoral, instituciones y políticas públicas, entre otros., Score: 6.8395534



### Queries semánticas

G. Consulta de grados relacionados con la análitica de datos

In [34]:
query_sentence = "analítica de datos"
query_vector = model.encode([query_sentence])[0]

parameters = {
     "field":"descripcion_vector",
     "query_vector": query_vector,
     "k":4,
     "num_candidates":100
}
res = es.search(
    index=index_name, 
    knn=parameters)

print("Search Results:")
for hit in res['hits']['hits']:
     print(f"Document ID: {hit['_id']}, Nombre Título: {hit['_source']['nombre']}, Description: {hit['_source']['descripcion']}, Score: {hit['_score']}")
     print("")

Search Results:
Document ID: 86, Nombre Título: Analítica de Negocio, Description: Este grado combina análisis de datos masivos con habilidades en negocios y tecnología. Se centra en identificar relaciones significativas en grandes conjuntos de datos. Los estudiantes aprenden a organizar, sistematizar y planificar para resolver problemas complejos, analizar datos cuantitativos y cualitativos, y tomar decisiones informadas. También se enfoca en habilidades blandas como liderazgo, trabajo en equipo, comunicación y pensamiento crítico., Score: 0.82298696

Document ID: 85, Nombre Título: Business Analytics, Description: Este grado combina administración, análisis estadístico y tecnología para analizar grandes cantidades de datos en el ámbito empresarial. Los estudiantes aprenden a identificar problemas, diseñar proyectos e informes digitales y tomar decisiones basadas en datos cuantitativos y cualitativos., Score: 0.8137814

Document ID: 84, Nombre Título: Análisis de Negocios, Description

H. Consulta de grados con salidas relacionadas con consultoría

In [36]:
query_sentence = "trabajo en consultoras"
query_vector = model.encode([query_sentence])[0]

parameters = {
     "field":"salidas_vector",
     "query_vector": query_vector,
     "k":4,
     "num_candidates":100
}
res = es.search(
    index=index_name, 
    knn=parameters)

print("Search Results:")
for hit in res['hits']['hits']:
     print(f"Document ID: {hit['_id']}, Nombre Título: {hit['_source']['nombre']}, Description: {hit['_source']['descripcion']}, Score: {hit['_score']}")
     print("")

Search Results:
Document ID: 92, Nombre Título: Administración de Empresas, Description: El grado aborda aspectos jurídicos y financieros en la gestión de empresas y organismos públicos o privados. Se forma a profesionales capaces de identificar iniciativas, gestionar empresas, resolver problemas de dirección y realizar asesoría empresarial., Score: 0.84587693

Document ID: 195, Nombre Título: Ingeniería en Sistemas Industriales, Description: El título de Grado en Ingeniería en Sistemas Industriales es un grado generalista que abarca áreas como Mecánica, Electrónica y Automática, Energía y Organización Industrial., Score: 0.82034004

Document ID: 124, Nombre Título: Diseño y Gestión de Proyectos Transmedia, Description: Aprenderás a gestionar proyectos que utilizan la narrativa transmedia para crear mundos e historias interactivas y motivadoras, con especialización en videojuegos y gamificación., Score: 0.8121705

Document ID: 80, Nombre Título: Bachelor in International Business Manag

I. Consulta de grados relacionados con la colaboración con el medioambiente y sostenibilidad

In [38]:
query_sentence = "colaborar con el medioambiente y la sostenibilidad"
query_vector = model.encode([query_sentence])[0]

parameters = {
     "field":"salidas_vector",
     "query_vector": query_vector,
     "k":4,
     "num_candidates":100
}
res = es.search(
    index=index_name, 
    knn=parameters)

print("Search Results:")
for hit in res['hits']['hits']:
     print(f"Document ID: {hit['_id']}, Nombre Título: {hit['_source']['nombre']}, Description: {hit['_source']['descripcion']}, Score: {hit['_score']}")
     print("")

Search Results:
Document ID: 183, Nombre Título: Ingeniería Ambiental, Description: El Grado en Ingeniería Ambiental busca comprender la interacción entre el ser humano y el medio ambiente. Los graduados deben diseñar tratamientos para mitigar los cambios ambientales causados por actividades industriales, equilibrando las necesidades materiales con el respeto al Medio Ambiente., Score: 0.8877063

Document ID: 182, Nombre Título: Ingeniería del Medio Natural © Comunidad de Madrid. Edición:, Description: Formar profesionales con formación científica y tecnológica para desarrollar tecnologías medioambientales y ecológicas en gestión, conservación y protección del Medio Natural., Score: 0.850938

Document ID: 136, Nombre Título: Ciencias Ambientales y Sostenibilidad, Description: Los estudios buscan proporcionar conocimientos para entender y proteger el medio ambiente, incluyendo la gestión ambiental y planificación territorial., Score: 0.80933297

Document ID: 109, Nombre Título: Estudios

J. Consulta de grados relacionados con ambiente sanitario

In [41]:
query_sentence = "ambiente sanitario"
query_vector = model.encode([query_sentence])[0]

parameters = {
     "field":"salidas_vector",
     "query_vector": query_vector,
     "k":4,
     "num_candidates":100
}
res = es.search(
    index=index_name, 
    knn=parameters)

print("Search Results:")
for hit in res['hits']['hits']:
     print(f"Document ID: {hit['_id']}, Nombre Título: {hit['_source']['nombre']}, Description: {hit['_source']['descripcion']}, Score: {hit['_score']}")
     print("")

Search Results:
Document ID: 215, Nombre Título: Ingeniería Informática Biomédica, Description: El grado en Ingeniería Informática Biomédica se enfoca en la aplicación de tecnología, informática y programación en salud y biomedicina., Score: 0.741694

Document ID: 155, Nombre Título: Fisioterapia, Description: La Fisioterapia es un estudio de técnicas terapéuticas físicas para tratar lesiones y enfermedades. Se enfoca en la rehabilitación y mejora de la autonomía del individuo, priorizando la colaboración con otros profesionales de la salud. Los fisioterapeutas utilizan técnicas manuales y no invasivas para identificar y solucionar problemas musculares, neurológicos y de columna vertebral., Score: 0.7404363

Document ID: 158, Nombre Título: Medicina, Description: La Medicina es la ciencia que se ocupa del diagnóstico, tratamiento y prevención de enfermedades. Los médicos trabajan en la promoción de la salud y el cuidado de los enfermos en hospitales y centros médicos., Score: 0.7395696

## Graph/Sparql

Primero presentamos nuestra ontología que utiliza los prefijos:
- sch: http://schema.org
- rdf: http://www.w3.org/1999/02/22-rdf-syntax-ns#
- xsd: http://www.w3.org/2001/XMLSchema#
- dat: http://example.org/universidadesMadrid#
- owl: http://www.w3.org/2002/07/owl#

![Diagrama de la ontología](grafo.png)

In [5]:
from SPARQLWrapper import SPARQLWrapper, JSON

# URL de tu endpoint SPARQL de GraphDB
endpoint_url = "http://graphdbdb:7200/repositories/Practica_GESTDB"

# Crear el objeto de conexión
sparql = SPARQLWrapper(endpoint_url)


A. Consultar todas las escuelas con sus universidades y datos de
contacto

In [6]:
query1 = """ 
PREFIX sch: <https://schema.org/>

SELECT ?schoolName ?universityName ?email ?phone ?address ?postalCode ?web
WHERE {
  ?school a sch:School ;
        sch:legalName ?schoolName ;
        sch:containedInPlace ?university .
        OPTIONAL { ?school sch:email ?email }
        OPTIONAL { ?school sch:telephone ?phone }
        OPTIONAL { ?school sch:address ?address }
        OPTIONAL { ?school sch:url ?web }

        ?university a sch:CollegeOrUniversity ;
}
ORDER BY ?universityName
LIMIT 10
"""

sparql.setQuery(query1)
sparql.setReturnFormat(JSON)

results = sparql.query().convert()

for r in results["results"]["bindings"]:
    school = r.get("schoolName", {}).get("value", "N/A")
    university = r.get("universityName", {}).get("value", "N/A")
    email = r.get("email", {}).get("value", "N/A")
    phone = r.get("phone", {}).get("value", "N/A")
    address = r.get("address", {}).get("value", "N/A")
    web = r.get("web", {}).get("value", "N/A")
    
    print(f"Facultad: {school} ; Universidad: {university} ; Email: {email} ; Telefono: {phone} ; Direccion: {address} ; Web: {web}")

Facultad: Facultad de Comunicacion y Artes Campus de MadridPrincesa ; Universidad: N/A ; Email: informa@nebrija.com ; Telefono: 91 452 11 03 ; Direccion: C/ Santa Cruz de Marcenado, 27 28015 Madrid ; Web: www.nebrija.com
Facultad: Facultad de Derecho y de Relaciones Internacionales Campus Madrid Princesa ; Universidad: N/A ; Email: informa@nebrija.com ; Telefono: 91 452 11 01 ; Direccion: C/ Santa Cruz de Marcenado, 27 28015 Madrid ; Web: www.nebrija.com
Facultad: Facultad de Ciencias Sociales Campus MadridPrincesa ; Universidad: N/A ; Email: informa@nebrija.com ; Telefono: 91 452 11 03 ; Direccion: C/ Santa Cruz de Marcenado, 27 28015 Madrid ; Web: www.nebrija.com
Facultad: Facultad de Ciencias Lenguas y Educacion Campus MadridPrincesa ; Universidad: N/A ; Email: informa@nebrija.com ; Telefono: 91 452 11 03 ; Direccion: C/ Santa Cruz de Marcenado, 27 28015 Madrid ; Web: www.nebrija.com
Facultad: Facultad de Comunicacion y Artes Campus MadridPrincesa ; Universidad: N/A ; Email: informa

B. Obtener la nota de corte más reciente de cada grado en el grupo 1 en la convocatoria ordinaria

In [9]:
query2 = """ 
PREFIX dat: <http://example.org/universidadesMadrid#>
PREFIX sch: <https://schema.org/>

SELECT ?university ?universityName ?programName (MAX(?year) AS ?latestYear) ?cutOff 
WHERE {
  ?cutOffScore a dat:CutOffScore ;
               dat:educational_program ?courseInstance ;
               dat:value ?cutOff ;
               dat:year ?year ;
               dat:call "ordinaria" ;
               dat:group "1".
               
  ?courseInstance sch:about ?program ;
                  sch:location ?school .
  ?program sch:name ?programName .
  
  ?school sch:containedInPlace ?university .
  ?university a sch:CollegeOrUniversity ;
              sch:legalName ?universityName .
}
GROUP BY ?university ?universityName ?program ?programName ?cutOff ?call ?group
ORDER BY ?programName
LIMIT 10

"""

sparql.setQuery(query2)
sparql.setReturnFormat(JSON)

results = sparql.query().convert()

for r in results["results"]["bindings"]:
    program_name = r.get("programName", {}).get("value", "N/A")
    latest_year = r.get("latestYear", {}).get("value", "N/A")
    cut_off = r.get("cutOff", {}).get("value", "N/A")
    
    print(f"Grado: {program_name} ; Año: {latest_year} ; Nota de corte: {cut_off}")

Grado: Administracion y Direccion de Empresas ; Año: 2025 ; Nota de corte: 12.01
Grado: Administracion y Direccion de Empresas ; Año: 2025 ; Nota de corte: 10.93
Grado: Administracion y Direccion de Empresas ; Año: 2025 ; Nota de corte: 10.71
Grado: Antropologia Social y Cultural ; Año: 2025 ; Nota de corte: 5.0
Grado: Antropologia Social y Cultural ; Año: 2025 ; Nota de corte: 5.0
Grado: Arqueologia ; Año: 2025 ; Nota de corte: 5.0
Grado: Arqueologia ; Año: 2025 ; Nota de corte: 5.0
Grado: Arquitectura Tecnica y Edificacion ; Año: 2025 ; Nota de corte: 7.71
Grado: Bellas Artes ; Año: 2025 ; Nota de corte: 11.69
Grado: Bellas Artes ; Año: 2025 ; Nota de corte: 9.94


C. Listar todas las universidades y las escuelas que contienen


In [10]:
query3 = """ 
PREFIX sch: <https://schema.org/>

SELECT ?universityName ?schoolName
WHERE {
  ?university a sch:CollegeOrUniversity ;
              sch:legalName ?universityName ;
              sch:containsPlace ?school .
  ?school sch:legalName ?schoolName .
}
ORDER BY ?universityName
LIMIT 10
"""

sparql.setQuery(query3)
sparql.setReturnFormat(JSON)

results = sparql.query().convert()

for r in results["results"]["bindings"]:
    university = r.get("universityName", {}).get("value", "N/A")
    school = r.get("schoolName", {}).get("value", "N/A")
    
    print(f"Universidad: {university} ; Facultad: {school}")

Universidad: Universidad Alfonso X El Sabio ; Facultad: Facultad de Ciencias de la Salud Campus de Villanueva de la Canada
Universidad: Universidad Alfonso X El Sabio ; Facultad: Facultad de Estudios Sociales y Lenguas Aplicadas Campus de Villanueva de la Canada
Universidad: Universidad Alfonso X El Sabio ; Facultad: Facultad de Educacion
Universidad: Universidad Alfonso X El Sabio ; Facultad: Facultad de Estudios Sociales y Lenguas Aplicadas
Universidad: Universidad Alfonso X El Sabio ; Facultad: Escuela Politecnica Superior Campus de Villanueva de la Canada
Universidad: Universidad Alfonso X El Sabio ; Facultad: Facultad de Musica y Artes Escenicas
Universidad: Universidad Alfonso X El Sabio ; Facultad: Facultad de Farmacia Campus de Villanueva de la Canada
Universidad: Universidad Alfonso X El Sabio ; Facultad: Facultad de Veterinaria Campus de Villanueva de la Canada
Universidad: Universidad Alfonso X El Sabio ; Facultad: Escuela Poltecnica Superior Campus de Villanueva de la Canad

D. Nota de corte promedio por área

In [12]:
query4 = """ 
PREFIX dat: <http://example.org/universidadesMadrid#>
PREFIX sch: <https://schema.org/>

SELECT ?categoryName (AVG(?value) AS ?avgCutOff)
WHERE {
  ?cut a dat:CutOffScore ;
       dat:value ?value ;
       dat:educational_program ?courseInstance .
       
  ?courseInstance sch:about ?program .
  ?program sch:occupationalCategory ?category .
  ?category sch:name ?categoryName .
}
GROUP BY ?categoryName
ORDER BY DESC(?avgCutOff)
LIMIT 10
"""

sparql.setQuery(query4)
sparql.setReturnFormat(JSON)

results = sparql.query().convert()

for r in results["results"]["bindings"]:
    category = r.get("categoryName", {}).get("value", "N/A")
    avg_cutoff = r.get("avgCutOff", {}).get("value", "N/A")
    
    print(f"Rama: {category} ; Nota de corte media: {avg_cutoff}")

Rama: Ingeniería Biomédica ; Nota de corte media: 10.860001
Rama: Psicología ; Nota de corte media: 10.82
Rama: Nutrición Humana y Dietética ; Nota de corte media: 9.725
Rama: Ingeniería Aeroespacial ; Nota de corte media: 9.622499
Rama: Ciencias de la Actividad Física y del Deporte ; Nota de corte media: 9.59
Rama: Paisajismo ; Nota de corte media: 9.4
Rama: Ingeniería Industrial ; Nota de corte media: 8.7775
Rama: Ingeniería Informática ; Nota de corte media: 8.363572
Rama: Ingeniería Civil ; Nota de corte media: 8.190001
Rama: Arquitectura ; Nota de corte media: 7.92


E. Ver todos los programas ofrecidos en una escuela concreta

In [13]:
query5 = """ 
PREFIX sch: <https://schema.org/>

SELECT ?schoolName ?programName ?categoryName ?universityName
WHERE {
  ?courseInstance a sch:CourseInstance ;
                  sch:about ?program ;
                  sch:location ?school .
  ?school sch:legalName ?schoolName ;
          sch:containedInPlace ?university .
  ?university sch:legalName ?universityName .
  ?program sch:name ?programName ;
           sch:occupationalCategory ?category .
  ?category sch:name ?categoryName .

  FILTER(STR(?universityName) = "Universidad Politécnica de Madrid")
  FILTER(CONTAINS(LCASE(STR(?schoolName)), "campus sur"))
}
ORDER BY ?programName
"""

sparql.setQuery(query5)
sparql.setReturnFormat(JSON)

results = sparql.query().convert()

for r in results["results"]["bindings"]:
    school = r.get("schoolName", {}).get("value", "N/A")
    university = r.get("universityName", {}).get("value", "N/A")
    program = r.get("programName", {}).get("value", "N/A")
    category = r.get("categoryName", {}).get("value", "N/A")
    
    print(f"Facultad: {school} ; Universidad: {university} ; Grado: {program} ; Área: {category}")

Facultad: Escuela Tecnica Superior de Ingenieria de Sistemas Informaticos Campus Sur ; Universidad: Universidad Politécnica de Madrid ; Grado: Ciencia de Datos e Inteligencia Artificial ; Área: Ingeniería Informática
Facultad: Escuela Tecnica Superior de Ingenieria de Sistemas de Telecomunicacion Campus Sur ; Universidad: Universidad Politécnica de Madrid ; Grado: Ingenieria Electronica de Comunicaciones ; Área: Ingeniería en Telecomunicación
Facultad: Escuela Tecnica Superior de Ingenieros en Topografia, Geodesia y Cartografia Campus Sur ; Universidad: Universidad Politécnica de Madrid ; Grado: Ingenieria Geomatica ; Área: Geomática
Facultad: Escuela Tecnica Superior de Ingenieria de Sistemas de Telecomunicacion Campus Sur ; Universidad: Universidad Politécnica de Madrid ; Grado: Ingenieria de Sonido e Imagen ; Área: Ingeniería en Telecomunicación
Facultad: Escuela Tecnica Superior de Ingenieros en Topografia, Geodesia y Cartografia Campus Sur ; Universidad: Universidad Politécnica de

# CONCLUSIONES

Los resultados obtenidos son claros y comprensibles. 

Las consultas SQL ofrecen información estructurada adecuada para obtener listados en formato tabla de los datos, pudiendo hacer relaciones y comparaciones entre grados, áreas, universidades, etc.

Las consultas de Elaticsearch son útiles cuando se quiere realizar una búsqueda más flexible o basada en similitud, especialmente las semánticas que permiten realizar consultas más genéricas.

Por último, las consultas SPARQL y el uso del modelo RDF son adecuados cuando se desea consultar información interrelacionada de forma más escalable, ya que permiten describir universidades, facultades, grados, ... como entidades conectadas. Esto permite realizar consultas complejas sin depender de una estructura tabular, aportando más flexibilidad. 

En conclusión, cada tuoi de consulta demuestra su utilidad según el tipo de análisis que se quiere realizar.